# **Homework 10 - Adversarial Attack**

## Enviroment & Download

We make use of [pytorchcv](https://pypi.org/project/pytorchcv/) to obtain CIFAR-10 pretrained model, so we need to set up the enviroment first. We also need to download the data (200 images) which we want to attack.

In [1]:
# set up environment
!pip install pytorchcv
!pip install imgaug

# download
!gdown --id 1t2UFQXr1cr5qLMBK2oN2rY1NDypi9Nyw --output data.zip

# if the above link isn't available, try this one
# !wget https://www.dropbox.com/s/lbpypqamqjpt2qz/data.zip

# unzip
!sudo apt-get update -y
!sudo apt-get install zip -y
!unzip ./data.zip

/home/u9941117/.local/lib/python3.9/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1t2UFQXr1cr5qLMBK2oN2rY1NDypi9Nyw
To: /home/u9941117/data.zip
100%|█████████████████████████████████████████| 490k/490k [00:00<00:00, 200MB/s]
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                   
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 2s (154 kB/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version

In [2]:
!rm ./data.zip

In [3]:
import torch
import torch.nn as nn
from pytorchcv.model_provider import get_model as ptcv_get_model
import random
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 2

def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		  torch.cuda.manual_seed(seed)
		  torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0) 

## Global Settings 
#### **[NOTE]**: Don't change the settings here, or your generated image might not meet the constraint.
* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [4]:
# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std

In [5]:
root = './data' # directory for storing benign images
# benign images: images which do not contain adversarial perturbations
# adversarial images: images which include adversarial perturbations

## Data

Construct dataset and dataloader from root directory. Note that we store the filename of each image for future usage.

In [6]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    def __getname__(self):
        return self.names
    def __len__(self):
        return len(self.images)

adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

print(f'number of images = {adv_set.__len__()}')

number of images = 200


## Utils -- Benign Images Evaluation

In [7]:
# to evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)

## Utils -- Attack Algorithm

In [8]:
# reference: https://github.com/as791/Adversarial-Example-Attack-and-Defense/blob/master/Adversarial_Example_(Attack_and_defense).ipynb

import torch
from torchvision import transforms

# perform fgsm attack
def fgsm(model, x, y, loss_fn, epsilon=epsilon):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = loss_fn(model(x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    grad = x_adv.grad.detach()
    x_adv = x_adv + epsilon * grad.sign()
    return x_adv

# alpha and num_iter can be decided by yourself
alpha = 0.8/255/std

def ifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20):
    x_adv = x.detach().clone()
    ################ TODO: Medium baseline #######################
    # write a loop with num_iter times
    for i in range(num_iter):
      # TODO: Each iteration, execute fgsm
      x_adv = x_adv.detach().clone() # initialize x_adv as original benign image x
      x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
      loss = loss_fn(model(x_adv), y) # calculate loss
      loss.backward() # calculate gradient
      # fgsm: use gradient ascent on x_adv to maximize loss
      grad = x_adv.grad.detach()
      x_adv = x_adv + alpha * grad.sign()
      #x_adv = torch.clamp(x_adv, 0, 1)
      x_adv = torch.clamp(x_adv, x-epsilon, x+epsilon)

    return x_adv

## Utils -- Attack
* Recall
  * ToTensor() can be seen as a function where $T(x) = x/255$.
  * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.

* Inverse function
  * Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
  * Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.

* Special Noted
  * ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
  * Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [9]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

## Model / Loss Function

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py). Please select models which has _cifar10 suffix. Other kinds of models are prohibited, and it will be considered to be cheating if you use them. 

Note: Some of the models cannot be accessed/loaded. You can safely skip them since TA's model will not use those kinds of models.

In [10]:
# This function is used to check whether you use models pretrained on cifar10 instead of other datasets
def model_checker(model_name):
  assert ('cifar10' in model_name) and ('cifar100' not in model_name), 'The model selected is not pretrained on cifar10!'

In [11]:
from pytorchcv.model_provider import get_model as ptcv_get_model

model_name = 'resnet110_cifar10'
model_checker(model_name)

model = ptcv_get_model(model_name, pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.95000, benign_loss = 0.22678


In [12]:
from pytorchcv.model_provider import get_model as ptcv_get_model
model_names = [
    'diaresnet20_cifar10',
    'diaresnet56_cifar10',
    'diaresnet110_cifar10',
    'diaresnet164bn_cifar10',
    'diapreresnet20_cifar10',
    'diapreresnet56_cifar10',
    'diapreresnet110_cifar10',
    'diapreresnet164bn_cifar10',
    'shakeshakeresnet20_2x16d_cifar10',
    'shakeshakeresnet26_2x32d_cifar10',
    'pyramidnet110_a48_cifar10',
    'pyramidnet110_a84_cifar10', 
    'pyramidnet110_a270_cifar10', 
    'pyramidnet164_a270_bn_cifar10', 
    'pyramidnet200_a240_bn_cifar10', 
    'pyramidnet236_a220_bn_cifar10', 
    'pyramidnet272_a200_bn_cifar10', 
    'densenet40_k12_cifar10',
    'densenet40_k12_bc_cifar10', 
    'densenet40_k24_bc_cifar10', 
    'densenet40_k36_bc_cifar10', 
    'densenet100_k12_cifar10',
    'densenet100_k24_cifar10', 
    'densenet100_k12_bc_cifar10', 
    'densenet190_k40_bc_cifar10', 
    'densenet250_k24_bc_cifar10', 
    'xdensenet40_2_k24_bc_cifar10', 
    'xdensenet40_2_k36_bc_cifar10', 
    'wrn16_10_cifar10',
    'wrn28_10_cifar10',
    'wrn40_8_cifar10',
    'wrn20_10_1bit_cifar10',
    'wrn20_10_32bit_cifar10', 
    'ror3_56_cifar10',
    'ror3_110_cifar10', 
    'ror3_164_cifar10',
    'rir_cifar10',
    'resnext29_32x4d_cifar10',
    'resnext29_16x64d_cifar10',
    'resnext272_1x64d_cifar10',
    'resnext272_2x32d_cifar10',
    'nin_cifar10',
    'resnet20_cifar10',
    'resnet56_cifar10',
    'resnet110_cifar10',
    'resnet164bn_cifar10',
    'resnet272bn_cifar10',
    'resnet542bn_cifar10',
    'resnet1001_cifar10',
    'resnet1202_cifar10',
    'preresnet20_cifar10',
    'preresnet56_cifar10',
    'preresnet110_cifar10',
    'preresnet164bn_cifar10',
    'preresnet272bn_cifar10',
    'preresnet542bn_cifar10',
    'preresnet1001_cifar10',
    'preresnet1202_cifar10',
    'seresnet20_cifar10',
    'seresnet56_cifar10',
    'seresnet110_cifar10',
    'seresnet164bn_cifar10',
    'seresnet272bn_cifar10',
    'seresnet542bn_cifar10', 
    'sepreresnet20_cifar10', 
    'sepreresnet56_cifar10', 
    'sepreresnet110_cifar10', 
    'sepreresnet164bn_cifar10', 
    'sepreresnet272bn_cifar10', 
    'sepreresnet542bn_cifar10', 
]
################ BOSS BASELINE ######################
class ensembleNet(nn.Module):
    def __init__(self, model_names):
        super().__init__()
        self.models = nn.ModuleList([ptcv_get_model(name, pretrained=True) for name in model_names])
        
    def forward(self, x):
        total_logits = 0
        for model in self.models:
            model_output = model(x.clone())
            total_logits += model_output

        averaged_logits = total_logits / len(self.models)
        return averaged_logits

for model_name in model_names:
  model_checker(model_name)

ensemble_model = ensembleNet(model_names).to(device)
ensemble_model.eval()

benign_acc, benign_loss = epoch_benign(ensemble_model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

/home/u9941117/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


benign_acc = 0.98000, benign_loss = 0.09205


## FGSM

In [13]:
adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(ensemble_model, adv_loader, fgsm, loss_fn)
print(f'fgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')

create_dir(root, 'fgsm', adv_examples, adv_names)

fgsm_acc = 0.57000, fgsm_loss = 1.96559


# IFGSM

In [14]:
adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(ensemble_model, adv_loader, ifgsm, loss_fn)
print(f'ifgsm_acc = {ifgsm_acc:.5f}, ifgsm_loss = {ifgsm_loss:.5f}')

create_dir(root, 'ifgsm', adv_examples, adv_names)

ifgsm_acc = 0.00500, ifgsm_loss = 13.84004


In [15]:
%cd ifgsm
!tar zcvf ../ifgsm.tgz *
%cd ..

/home/u9941117/ifgsm
airplane/
airplane/airplane17.png
airplane/airplane7.png
airplane/airplane16.png
airplane/airplane12.png
airplane/airplane13.png
airplane/airplane20.png
airplane/airplane19.png
airplane/airplane5.png
airplane/airplane14.png
airplane/airplane4.png
airplane/airplane15.png
airplane/airplane9.png
airplane/airplane1.png
airplane/airplane6.png
airplane/airplane10.png
airplane/airplane8.png
airplane/airplane11.png
airplane/airplane18.png
airplane/airplane3.png
airplane/airplane2.png
automobile/
automobile/automobile17.png
automobile/automobile12.png
automobile/automobile18.png
automobile/automobile5.png
automobile/automobile9.png
automobile/automobile14.png
automobile/automobile7.png
automobile/automobile20.png
automobile/automobile19.png
automobile/automobile4.png
automobile/automobile13.png
automobile/automobile1.png
automobile/automobile2.png
automobile/automobile6.png
automobile/automobile3.png
automobile/automobile10.png
automobile/automobile8.png
automobile/automobi

In [16]:
#from google.colab import files
files.download('ifgsm.tgz')

NameError: name 'files' is not defined

## Visualization

In [ ]:
import matplotlib.pyplot as plt

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10, 20))
cnt = 0
for i, cls_name in enumerate(classes):
    path = f'{cls_name}/{cls_name}1.png'
    # benign image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./data/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'benign: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
    # adversarial image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./fgsm/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'adversarial: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
plt.tight_layout()
plt.show()

## Report Question
* Make sure you follow below setup: the source model is "resnet110_cifar10", applying the vanilla fgsm attack on `dog2.png`. You can find the perturbed image in `fgsm/dog2.png`.

In [ ]:
# original image
path = f'dog/dog2.png'
im = Image.open(f'./data/{path}')
logit = model(transform(im).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'benign: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')
plt.imshow(np.array(im))
plt.tight_layout()
plt.show()

# adversarial image 
adv_im = Image.open(f'./fgsm/{path}')
logit = model(transform(adv_im).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')
plt.imshow(np.array(adv_im))
plt.tight_layout()
plt.show()

## Passive Defense - JPEG compression
JPEG compression by imgaug package, compression rate set to 70

Reference: https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.JpegCompression

Note: If you haven't implemented the JPEG compression, this module will return an error. Don't worry about this.

In [ ]:
import imgaug.augmenters as iaa

# pre-process image
x = transforms.ToTensor()(adv_im)*255
x = x.permute(1, 2, 0).numpy()
x = x.astype(np.uint8)

# TODO: use "imgaug" package to perform JPEG compression (compression rate = 70)
# compressed_x =  ... x .. 
# Reference: https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.JpegCompression
aug = iaa.JpegCompression(compression=(70,70))
compressed_x = aug(images=x)

logit = model(transform(compressed_x).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'JPEG adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')


plt.imshow(compressed_x)
plt.tight_layout()
plt.show()